In [13]:
#Importing Libraries
import numpy as np
import pandas as pd
import pickle
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from functools import reduce
from pyspark.sql.types import ArrayType, StringType
import re

In [25]:
from IPython.core.display import HTML,display
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [64]:
jdf = [
  {
    "upi_transaction_id": "YBL3b55ac093d76488c9cef1425265396cd",
    "transaction_type": "PAY",
    "transaction_reference_id": "1200233635",
    "customer_reference_number": 203230251039,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 75000,
    "payee_account_number": "918020110872063",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000000",
    "payee_name": "PAVAN BAR",
    "payee_vpa": "gpay-11200233635@okbizaxis",
    "payee_amount": 75000,
    "state": "SUCCESS",
    "created": "2022-02-01T19:58:44.300Z",
    "updated": "2022-02-01T19:58:46.885Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBLdc58fbba10be465e8991493bf6632ed8",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202011229232764503394",
    "customer_reference_number": 203272206073,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 1800,
    "payee_account_number": "00000054007513786",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "SBIN0040605",
    "payee_name": "SANJEEVA  C",
    "payee_vpa": "Q513055970@ybl",
    "payee_amount": 1800,
    "state": "SUCCESS",
    "created": "2022-02-01T12:29:25.697Z",
    "updated": "2022-02-01T12:29:29.521Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBLdbc43d56f6764418857167defc91ef16",
    "transaction_type": "PAY",
    "transaction_reference_id": "1200025300",
    "customer_reference_number": 200777647233,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 9000,
    "payee_account_number": "918020110872063",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000000",
    "payee_name": "G N STORES",
    "payee_vpa": "gpay-11200025300@okbizaxis",
    "payee_amount": 9000,
    "state": "SUCCESS",
    "created": "2022-01-07T12:18:23.183Z",
    "updated": "2022-01-07T12:18:28.495Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL6f7be574f7a84d37ac6f32f040ffabb4",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202030848314044720230",
    "customer_reference_number": 203481181335,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 1800,
    "payee_account_number": "00000054007513786",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "SBIN0040605",
    "payee_name": "SANJEEVA  C",
    "payee_vpa": "Q513055970@ybl",
    "payee_amount": 1800,
    "state": "SUCCESS",
    "created": "2022-02-03T08:48:35.539Z",
    "updated": "2022-02-03T08:48:37.328Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL36518a861bf440cba731b9929db8eabf",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202032215493445088110",
    "customer_reference_number": 203423441798,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 200000,
    "payee_account_number": "5445715243",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "KKBK0008062",
    "payee_name": "ASHOK   K",
    "payee_vpa": "9986528460@ybl",
    "payee_amount": 200000,
    "state": "SUCCESS",
    "created": "2022-02-03T22:15:52.653Z",
    "updated": "2022-02-03T22:15:54.833Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL98cd64615a3842939a0dd6f6277b75a6",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202252126498206823913",
    "customer_reference_number": 205647631673,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 40000,
    "payee_account_number": "15042600001897",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "CNRB0000033",
    "payee_name": "DARSHAN D V",
    "payee_vpa": "darshan.dv@ybl",
    "payee_amount": 40000,
    "state": "SUCCESS",
    "created": "2022-02-25T21:26:52.638Z",
    "updated": "2022-02-25T21:26:54.200Z",
    "data_dt": "2022-02-27T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBLbf83393e82e4454f991b4e79c5445655",
    "transaction_type": "PAY",
    "transaction_reference_id": "1197405594",
    "customer_reference_number": 203134457255,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 1800,
    "payee_account_number": "918020110872063",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000000",
    "payee_name": "Paan Stall",
    "payee_vpa": "gpay-11197405594@okbizaxis",
    "payee_amount": 1800,
    "state": "SUCCESS",
    "created": "2022-01-31T14:06:25.823Z",
    "updated": "2022-01-31T14:06:28.077Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL0ccf09ef9a52416cb4135bc4e9fc4b83",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202060935527432168012",
    "customer_reference_number": 203728933198,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 4700,
    "payee_account_number": "00000054007513786",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "SBIN0040605",
    "payee_name": "SANJEEVA  C",
    "payee_vpa": "Q513055970@ybl",
    "payee_amount": 4700,
    "state": "SUCCESS",
    "created": "2022-02-06T09:35:55.546Z",
    "updated": "2022-02-06T09:35:57.076Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL7aad4fb33d9e4345a4d8dc9becf945d3",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202071612588899207083",
    "customer_reference_number": 203839653851,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 5000,
    "payee_account_number": "4219101002468",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "CNRB0000000",
    "payee_name": "VIJAY C M",
    "payee_vpa": "8197534210@ybl",
    "payee_amount": 5000,
    "state": "SUCCESS",
    "created": "2022-02-07T16:13:01.681Z",
    "updated": "2022-02-07T16:13:04.356Z",
    "data_dt": "2022-02-07T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL558f8ca626634ced9e51babad7f133ea",
    "transaction_type": "PAY",
    "transaction_reference_id": "1173717007",
    "customer_reference_number": 204815288220,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100516700,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "H VIGNESH",
    "payer_vpa": "7019270274@ybl",
    "payer_amount": 1500,
    "payee_account_number": "918020110872063",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000000",
    "payee_name": "A1 Plantain Mandy",
    "payee_vpa": "gpay-11173717007@okbizaxis",
    "payee_amount": 1500,
    "state": "SUCCESS",
    "created": "2022-02-17T20:51:01.473Z",
    "updated": "2022-02-17T20:51:03.110Z",
    "data_dt": "2022-02-19T00:00:00.000Z",
    "phone_number": 917019270274,
    "account_number": 9542500100516700,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "H VIGNESH",
    "mbanking_enabled": 1,
    "registered_at_acct": "2021-11-25T11:36:37.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBLa69c2955ef8c4a68a2b8745db7191dd1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201041657290521255407",
    "customer_reference_number": 200447386801,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 1100000,
    "payee_account_number": "520191054361520",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "CORP0000098",
    "payee_name": "JOVESTER DSOUZA",
    "payee_vpa": "9964507166@ybl",
    "payee_amount": 1100000,
    "state": "SUCCESS",
    "created": "2022-01-04T16:57:33.524Z",
    "updated": "2022-01-04T16:57:37.698Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL9da71b83cde64a12a7ded6521fe15470",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2202101904307438127077",
    "customer_reference_number": 204111617373,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 26000,
    "payee_account_number": "0161301000057472",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "LAVB0000161",
    "payee_name": "KALU RAM",
    "payee_vpa": "Q93659325@ybl",
    "payee_amount": 26000,
    "state": "SUCCESS",
    "created": "2022-02-10T19:04:35.591Z",
    "updated": "2022-02-10T19:04:38.689Z",
    "data_dt": "2022-02-12T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "YBL2a3a489ee99e44caa56910bc3cc110d1",
    "transaction_type": "PAY",
    "transaction_reference_id": "T2201061024416257077813",
    "customer_reference_number": 200680911557,
    "note": "Payment from PhonePe",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 72000,
    "payee_account_number": "149110100033175",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "ANDB0001491",
    "payee_name": "SHANKARLINGAPPA I HUGAR",
    "payee_vpa": "9008819966@ybl",
    "payee_amount": 72000,
    "state": "SUCCESS",
    "created": "2022-01-06T10:24:45.075Z",
    "updated": "2022-01-06T10:24:47.867Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  },
  {
    "upi_transaction_id": "APLe2773aa50c1847a7b2b7233aa9223da6",
    "transaction_type": "COLLECT",
    "transaction_reference_id": "TQhuK1uH4fZUdaE6GhNFsVsxka2j1mrRasG",
    "customer_reference_number": 201115582828,
    "note": "You are paying for an Amazon order",
    "payer_account_number": 9542500100711100,
    "payer_account_type": "SAVINGS",
    "payer_ifsc": "KARB0000954",
    "payer_name": "PHILOMINA",
    "payer_vpa": "8073789525@ybl",
    "payer_amount": 104700,
    "payee_account_number": "10001291013360",
    "payee_account_type": "SAVINGS",
    "payee_ifsc": "UTIB0000100",
    "payee_name": "AMAZON",
    "payee_vpa": "amazon@apl",
    "payee_amount": 104700,
    "state": "SUCCESS",
    "created": "2022-01-11T15:01:13.297Z",
    "updated": "2022-01-11T15:01:34.397Z",
    "data_dt": "2022-01-31T00:00:00.000Z",
    "phone_number": 918073789525,
    "account_number": 9542500100711100,
    "account_type": "SAVINGS",
    "account_ifsc": "KARB0000954",
    "account_holder_name": "PHILOMINA",
    "mbanking_enabled": 0,
    "registered_at_acct": "2020-01-10T20:03:30.000Z",
    "Category1": "Personal Transfer",
    "Category2": "TRANSFER_FRIENDS_AND_FAMILY"
  }]

In [28]:
# Example usage
input_data_1 = [{
    'payer_name': 'John Doe',
    'payer_vpa': 'john.doe@bank',
    'payee_account_type': 'savings',
    'payee_name': 'Jane Smith',
    'payee_vpa': 'jane.smith@bank',
    'payer_account_type': 'current',
    "type":"savings",
    "Mode":"UPI"
},
                {
    'payer_name': 'John Doe',
    'payer_vpa': 'john.doe@bank',
    'payee_account_type': 'savings',
    'payee_name': 'Jane Smith',
    'payee_vpa': 'jane.smith@bank',
    'payer_account_type': 'current',
    "type":"savings",
    "Mode":"UPI"
    }]

In [65]:
df = spark.read.json(sc.parallelize([jdf]))
df.show(truncate=False)

+-----------------+---------------------------+-------------------+------------+----------------+------------+------------------------+-------------------------+------------------------+----------------+----------------------------------+--------------------+------------------+------------+-----------+-----------------------+--------------------------+--------------------+------------------+------------+-----------+----------+--------------+------------+------------------------+-------+-----------------------------------+----------------+------------------------+-----------------------------------+
|Category1        |Category2                  |account_holder_name|account_ifsc|account_number  |account_type|created                 |customer_reference_number|data_dt                 |mbanking_enabled|note                              |payee_account_number|payee_account_type|payee_amount|payee_ifsc |payee_name             |payee_vpa                 |payer_account_number|payer_account_type|

In [72]:
cols = ("Category1","Category2")

df2 = df.drop(*cols)

In [73]:
df2.show(truncate=False)

+-------------------+------------+----------------+------------+------------------------+-------------------------+------------------------+----------------+----------------------------------+--------------------+------------------+------------+-----------+-----------------------+--------------------------+--------------------+------------------+------------+-----------+----------+--------------+------------+------------------------+-------+-----------------------------------+----------------+------------------------+-----------------------------------+
|account_holder_name|account_ifsc|account_number  |account_type|created                 |customer_reference_number|data_dt                 |mbanking_enabled|note                              |payee_account_number|payee_account_type|payee_amount|payee_ifsc |payee_name             |payee_vpa                 |payer_account_number|payer_account_type|payer_amount|payer_ifsc |payer_name|payer_vpa     |phone_number|registered_at_acct      |sta

In [74]:
# Define the custom tokenizer
def custom_tokenizer(text):
    pattern = re.compile(r'[a-zA-Z]+\d+')
    return pattern.findall(text)

# Load the saved models and vectorizers
clf_cat1 = pickle.load(open("pickle_letest/classifier1.pkl", "rb"))
clf_cat2 = pickle.load(open("pickle_letest/classifier2.pkl", "rb"))
tfidf_payer_name = pickle.load(open("pickle_letest/payer_name.pkl", "rb"))
tfidf_payee_name = pickle.load(open("pickle_letest/payee_name.pkl", "rb"))
tfidf_payee_account_type = pickle.load(open("pickle_letest/payee_account_type.pkl", "rb"))
tfidf_payer_account_type = pickle.load(open("pickle_letest/payer_account_type.pkl", "rb"))
tfidf_payer_vpa = pickle.load(open("pickle_letest/payer_vpa.pkl", "rb"))
tfidf_payee_vpa = pickle.load(open("pickle_letest/payee_vpa.pkl", "rb"))



# Define the prediction function
def predict_categories(payer_name, payee_name, payee_account_type,
                       payer_account_type, payer_vpa, payee_vpa):
    # Transform input data using the TFIDF vectorizers
    payer_name_vec = tfidf_payer_name.transform([payer_name])
    payee_name_vec = tfidf_payee_name.transform([payee_name])
    payee_account_type_vec = tfidf_payee_account_type.transform([payee_account_type])
    payer_account_type_vec = tfidf_payer_account_type.transform([payer_account_type])
    payer_vpa_vec = tfidf_payer_vpa.transform([payer_vpa])
    payee_vpa_vec = tfidf_payee_vpa.transform([payee_vpa])

    tfidf_matrix = pd.concat([pd.DataFrame(payer_name_vec.toarray()),
                              pd.DataFrame(payee_name_vec.toarray()),
                              pd.DataFrame(payee_account_type_vec.toarray()),
                              pd.DataFrame(payer_account_type_vec.toarray()),
                              pd.DataFrame(payer_vpa_vec.toarray()),
                              pd.DataFrame(payee_vpa_vec.toarray())], axis=1)

    # Predict
    prediction_cat1 = clf_cat1.predict(tfidf_matrix)
    prediction_cat2 = clf_cat2.predict(tfidf_matrix)

    return [prediction_cat1[0], prediction_cat2[0]]

# Register the prediction function as a UDF
predict_udf = udf(predict_categories, ArrayType(StringType()))

# Make sure your DataFrame has columns: payer_name, payee_name, payee_account_type, payer_account_type, payer_vpa, payee_vpa
result_df = df2.withColumn("predictions", predict_udf("payer_name", "payee_name", "payee_account_type", "payer_account_type", "payer_vpa", "payee_vpa"))
# Assuming result_df is the DataFrame with the combined predictions column
result_df = result_df.withColumn("category_level1", result_df["predictions"].getItem(0))
result_df = result_df.withColumn("category_level2", result_df["predictions"].getItem(1))

# Dropping the combined predictions column
result_df = result_df.drop("predictions")

# Show the resulting DataFrame
result_df.show()

+-------------------+------------+----------------+------------+--------------------+-------------------------+--------------------+----------------+--------------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+------------------+------------+-----------+----------+--------------+------------+--------------------+-------+------------------------+----------------+--------------------+--------------------+-----------------+--------------------+
|account_holder_name|account_ifsc|  account_number|account_type|             created|customer_reference_number|             data_dt|mbanking_enabled|                note|payee_account_number|payee_account_type|payee_amount| payee_ifsc|          payee_name|           payee_vpa|payer_account_number|payer_account_type|payer_amount| payer_ifsc|payer_name|     payer_vpa|phone_number|  registered_at_acct|  state|transaction_reference_id|transaction_type|             u

In [75]:
# Show the resulting DataFrame
result_df.show(truncate=False)

+-------------------+------------+----------------+------------+------------------------+-------------------------+------------------------+----------------+----------------------------------+--------------------+------------------+------------+-----------+-----------------------+--------------------------+--------------------+------------------+------------+-----------+----------+--------------+------------+------------------------+-------+-----------------------------------+----------------+------------------------+-----------------------------------+-----------------+---------------------------+
|account_holder_name|account_ifsc|account_number  |account_type|created                 |customer_reference_number|data_dt                 |mbanking_enabled|note                              |payee_account_number|payee_account_type|payee_amount|payee_ifsc |payee_name             |payee_vpa                 |payer_account_number|payer_account_type|payer_amount|payer_ifsc |payer_name|payer_vpa 